In [153]:
# Módulos
import ee
import folium
import geehydro

In [154]:
# Inicio de API
ee.Initialize()

#### Función generateGreedForGeometry

In [155]:
#funcion para la generacion de grillas
def generateGreedForGeometry(geom,dx,dy):
    """
    Función para generar grillas partiendo de una zona geográfica.
    """

    bounds = ee.Geometry(geom).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    xx = ee.List.sequence(xmin, xmax, dx)
    yy = ee.List.sequence(ymin, ymax, dy)

    #************************************
    def contYYa(y,x,dx,dy):

        x1 = ee.Number(x)
        x2 = ee.Number(x).add(ee.Number(dx))
        y1 = ee.Number(y)
        y2 = ee.Number(y).add(ee.Number(dy))

        coords = ee.List([x1, y1, x2, y2])
        rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)

        return ee.Feature(rect)

    def contYY(x):
        return yy.map(lambda yi: contYYa(y=yi,x=x,dx=dx,dy=dy) )
    #************************************

    cells = xx.map(contYY).flatten() 

    return ee.FeatureCollection(cells).filterBounds(geom)#.map(lambda c: ee.Feature(c).intersection(geom) )

#### Ejemplo

In [156]:
#----------------------------------------------
# Ejemplo
#----------------------------------------------

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Canaima National Park'#'Guatopo'
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))


# geometria
state = polygon.geometry()
dx = 0.09
dy = 0.09

# creando grilla
greed = generateGreedForGeometry(state,dx,dy)
centroide = state.centroid().coordinates().getInfo()[::-1]

#### Creando centroides

In [157]:
def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)


centroids = greed.map(getCentroid)

print('área del parque Km2: ', 0.1*polygon.first().geometry().area().divide(100*100).getInfo() ) 
print('área de cuadricula Km2: ', 0.1*greed.first().geometry().area().divide(100*100).getInfo() )
print('número de centroides: ', centroids.size().getInfo())

área del parque Km2:  289542.4562322496
área de cuadricula Km2:  997.0842463804966
número de centroides:  346


In [158]:
Map = folium.Map(location=centroide)
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Grids')
Map

#### Información Indices de vegetacion

In [159]:
# MODIS MOD13Q1
modis = ee.ImageCollection('MODIS/006/MOD13Q1')
modis = modis.filterDate(ee.DateRange('2019-12-01','2019-12-31'))

# select EVI and NDVI
evi = modis.select('EVI')
ndvi = modis.select('NDVI')

In [160]:
def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])
# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)

In [161]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')
Map.addLayer(scaled_ndvi.mean().clip(polygon)
            ,name='NDVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['red', 'yellow','green']}) 
Map.addLayer(scaled_evi.mean().clip(polygon)
            ,name='EVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['red', 'yellow','green']}) 
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Centroids')
Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map